In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from google.colab import files
uploaded = files.upload()

Saving clothes_features_combined.csv to clothes_features_combined.csv


In [3]:
clothes_df = pd.read_csv('clothes_features_combined.csv')
clothes_df.head()

,gender,product_category,product_name,details,category_type,combined_features
0,women,SKIRTS,z denim midi skirt,fade highwaist skirt feature fivepocket design...,Bottom,denim denim skirt midi
1,women,SKIRTS,faux leather mini skirt chain,highwaist mini skirt lining metal chain link i...,Bottom,faux leather skirt mini
2,women,SKIRTS,skort pocket,midwaist skort adjustable elasticate waistband...,Bottom,skort
3,women,SKIRTS,long satin skirt,long skirt high elastic waist,Bottom,satin skirt long
4,women,SKIRTS,knot midi skirt,highwaist flow midi skirt knot detail slit hem...,Bottom,skirt midi


# Preprocessing

In [4]:
for cat in clothes_df["category_type"].unique():
    var_name = cat.replace(" ", "_").lower() + "_df"
    globals()[var_name] = clothes_df[clothes_df["category_type"] == cat].copy()

In [5]:
accessories_df

,gender,product_category,product_name,details,category_type,combined_features
1125,women,ACCESSORIES,crossbody bag,crossbody bag adjustable strap lining zip clos...,Accessories,bag
1136,women,ACCESSORIES,tote interior bag,tote bag shoulder strap inner bag gather draws...,Accessories,bag tote
1142,women,ACCESSORIES,leather belt,leather belt fasten metal buckle belt loop,Accessories,leather belt
1171,women,ACCESSORIES,metal necklace,rigid metal necklace,Accessories,necklace
1172,women,ACCESSORIES,pack hoop earring,metal earring click fasten metal earring rhine...,Accessories,earring
...,...,...,...,...,...,...
3620,men,ACCESSORIES,baseball training cap,peak cap adjustable strap backzara athleticz,Accessories,cap
3622,men,ACCESSORIES,training short,training short lightweight stretchy technical ...,Accessories,short lightweight short
3624,men,ACCESSORIES,strappy cycling culotte,culotte highly stretchy fabric elastic strap f...,Accessories,NaN
3625,men,ACCESSORIES,cap,cap stretchy breathable fabric secure fit inne...,Accessories,cap breathable


In [6]:
bottom_df.head()

,gender,product_category,product_name,details,category_type,combined_features
0,women,SKIRTS,z denim midi skirt,fade highwaist skirt feature fivepocket design...,Bottom,denim denim skirt midi
1,women,SKIRTS,faux leather mini skirt chain,highwaist mini skirt lining metal chain link i...,Bottom,faux leather skirt mini
2,women,SKIRTS,skort pocket,midwaist skort adjustable elasticate waistband...,Bottom,skort
3,women,SKIRTS,long satin skirt,long skirt high elastic waist,Bottom,satin skirt long
4,women,SKIRTS,knot midi skirt,highwaist flow midi skirt knot detail slit hem...,Bottom,skirt midi


In [7]:
shoes_df.shape

(479, 6)

In [8]:
top_df.shape

(1620, 6)

# Zero-Shot Classification

In [9]:
#load the model
import torch
from transformers import pipeline
import tensorflow as tf

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [10]:
#candidate labels
labels = ["5-10°C cold weather", "10-15°C cool weather", "15-20°C mild weather", "20-25°C warm weather", "25-30°C hot weather", "rain"]

### Classifying Accessories Category

In [11]:
#zero-shot-classification on accessories category
accessories_df = accessories_df.fillna("").astype(str)
#accessories_df_copy = accessories_df[accessories_df['combined_features'].str.strip() != ""].copy()
accessories_df['text'] = accessories_df['combined_features'] + ' ' + accessories_df['product_name']
texts = accessories_df['text'].tolist()

batch_size = 16
results = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This outfit is designed to be worn in {} conditions.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

accessories_df['weather_label'] = results
accessories_df.head()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,gender,product_category,product_name,details,category_type,combined_features,text,weather_label
1125,women,ACCESSORIES,crossbody bag,crossbody bag adjustable strap lining zip clos...,Accessories,bag,bag crossbody bag,10-15°C cool weather
1136,women,ACCESSORIES,tote interior bag,tote bag shoulder strap inner bag gather draws...,Accessories,bag tote,bag tote tote interior bag,10-15°C cool weather
1142,women,ACCESSORIES,leather belt,leather belt fasten metal buckle belt loop,Accessories,leather belt,leather belt leather belt,15-20°C mild weather
1171,women,ACCESSORIES,metal necklace,rigid metal necklace,Accessories,necklace,necklace metal necklace,10-15°C cool weather
1172,women,ACCESSORIES,pack hoop earring,metal earring click fasten metal earring rhine...,Accessories,earring,earring pack hoop earring,15-20°C mild weather


In [17]:
accessories_df.to_csv("classified_accessories.csv", index=False)

### Classifying Shoes Category

In [21]:
#zero-shot-classification on shoes category
shoes_df = shoes_df.fillna("").astype(str)

shoes_df['text'] = shoes_df['combined_features'] + ' ' + shoes_df['product_name']
texts = shoes_df['text'].tolist()

batch_size = 16
results = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This outfit is designed to be worn in {} conditions.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

shoes_df['weather_label'] = results
shoes_df.head()

,gender,product_category,product_name,details,category_type,combined_features,text,weather_label
1137,women,SHOES,ballet flat buckle,ballet flat buckle strap instep round toesole ...,Shoes,,ballet flat buckle,10-15°C cool weather
1148,women,SHOES,flat leather cage sandal,flat leather cage sandal wide strap buckle ank...,Shoes,leather sandal,leather sandal flat leather cage sandal,15-20°C mild weather
1158,women,SHOES,leather mule loafer buckle,leather flat mule loafer metallic buckle detai...,Shoes,leather loafer mule,leather loafer mule leather mule loafer buckle,15-20°C mild weather
1163,women,SHOES,leather cowboy ankle boot,leather cowboystyle ankle boot block heel pull...,Shoes,leather boot,leather boot leather cowboy ankle boot,10-15°C cool weather
1175,women,SHOES,tassel loafer,flat loafer metallic buckle detail tassel fron...,Shoes,loafer,loafer tassel loafer,15-20°C mild weather


In [25]:
shoes_df.to_csv("classified_shoes.csv", index=False)
files.download("classified_shoes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Classifying Tops Category

In [26]:
#zero-shot-classification on top category
top_df = top_df.fillna("").astype(str)

top_df['text'] = top_df['combined_features'] + ' ' + top_df['product_name']
texts = top_df['text'].tolist()

batch_size = 16
results = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This outfit is designed to be worn in {} conditions.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

top_df['weather_label'] = results
top_df.head()

,gender,product_category,product_name,details,category_type,combined_features,text,weather_label
59,women,SHIRTS,poplin shirt,collar vneck shirt long sleeve buttonup,Top,poplin shirt long,poplin shirt long poplin shirt,15-20°C mild weather
60,women,SHIRTS,poplin shirt,shirt johnny collar long sleeve buttonup,Top,poplin shirt long,poplin shirt long poplin shirt,15-20°C mild weather
61,women,SHIRTS,cotton blend oxford shirt,shirt cotton blend johnny collar long sleeve a...,Top,cotton oxford shirt long,cotton oxford shirt long cotton blend oxford s...,15-20°C mild weather
62,women,SHIRTS,oxford shirt,shirt cotton blend johnny collar long sleeve b...,Top,cotton oxford shirt long,cotton oxford shirt long oxford shirt,15-20°C mild weather
63,women,SHIRTS,stripe oversize shirt,loosefitte shirt feature johnny collar long sl...,Top,shirt long,shirt long stripe oversize shirt,15-20°C mild weather


In [32]:
top_df.to_csv("classified_top.csv", index=False)
files.download("classified_top.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Classifying Bottoms Category

In [33]:
#zero-shot-classification on bottom category
bottom_df = bottom_df.fillna("").astype(str)

bottom_df['text'] = bottom_df['combined_features'] + ' ' + bottom_df['product_name']
texts = bottom_df['text'].tolist()

batch_size = 16
results = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This clothing is suitable for {} weather.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

bottom_df['weather_label'] = results
bottom_df.head()

,gender,product_category,product_name,details,category_type,combined_features,text,weather_label
0,women,SKIRTS,z denim midi skirt,fade highwaist skirt feature fivepocket design...,Bottom,denim denim skirt midi,denim denim skirt midi z denim midi skirt,10-15°C cool weather
1,women,SKIRTS,faux leather mini skirt chain,highwaist mini skirt lining metal chain link i...,Bottom,faux leather skirt mini,faux leather skirt mini faux leather mini skir...,10-15°C cool weather
2,women,SKIRTS,skort pocket,midwaist skort adjustable elasticate waistband...,Bottom,skort,skort skort pocket,5-10°C cold weather
3,women,SKIRTS,long satin skirt,long skirt high elastic waist,Bottom,satin skirt long,satin skirt long long satin skirt,10-15°C cool weather
4,women,SKIRTS,knot midi skirt,highwaist flow midi skirt knot detail slit hem...,Bottom,skirt midi,skirt midi knot midi skirt,10-15°C cool weather


In [39]:
bottom_df.to_csv("classified_bottom.csv", index=False)
files.download("classified_bottom.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Recommendation

In [40]:
temperature_mapping = {
            (5, 10): "5-10°C cold weather",
            (10, 15): "10-15°C cool weather",
            (15, 20): "15-20°C mild weather",
            (20, 25): "20-25°C warm weather",
            (25, 30): "25-30°C hot weather"
        }

In [41]:
def temperature_to_label(temperature):
        """
        Convert numeric temperature to label
        """
        try:
            temp = float(temperature)
            for (low, high), label in temperature_mapping.items():
                if low <= temp <= high:
                    return label
            if temp < 5:
                return "5-10°C cold weather"
            elif temp > 30:
                return "25-30°C hot weather"
        except ValueError:
            raise ValueError(f"Invalid temperature: {temperature}. Please enter 5-30.")

In [42]:
temperature = '25'
temperature_label = temperature_to_label(temperature)
temp_range = temperature_label.split(' ')[0]

In [51]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from TOP category

if len(top_df) > sample_size:
    sample_df = top_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(top_df)} total Top items")
else:
    sample_df = top_df
    print(f"🎯 Using all {len(sample_df)} top items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
top_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    top_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
top_recommendations[0]['product_name']

🎯 Sampling 5 random items from 1620 total Top items
⚡ Classifying sampled items in one batch...


'faux leather bomber jacket'

In [52]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from BOTTOM category

if len(bottom_df) > sample_size:
    sample_df = bottom_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(bottom_df)} total bottom items")
else:
    sample_df = bottom_df
    print(f"🎯 Using all {len(sample_df)} bottom items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
bottom_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    bottom_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
bottom_recommendations[0]['product_name']

🎯 Sampling 5 random items from 599 total bottom items
⚡ Classifying sampled items in one batch...


'straight fit trouser button'

In [53]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from SHOES category

if len(shoes_df) > sample_size:
    sample_df = shoes_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(shoes_df)} total shoes items")
else:
    sample_df = shoes_df
    print(f"🎯 Using all {len(sample_df)} shoes items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
shoes_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    shoes_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
shoes_recommendations[0]['product_name']

🎯 Sampling 5 random items from 479 total shoes items
⚡ Classifying sampled items in one batch...


'multipiece hightop sneaker'

In [54]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from ACCESSORIES category

if len(accessories_df) > sample_size:
    sample_df = accessories_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(accessories_df)} total accessories items")
else:
    sample_df = accessories_df
    print(f"🎯 Using all {len(sample_df)} accessories items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
accessories_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    accessories_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
accessories_recommendations[0]['product_name']

🎯 Sampling 5 random items from 181 total accessories items
⚡ Classifying sampled items in one batch...


'pack noshow sock'

In [55]:
labels

['5-10°C cold weather',
 '10-15°C cool weather',
 '15-20°C mild weather',
 '20-25°C warm weather',
 '25-30°C hot weather',
 'rain']

In [56]:
# Save to a JSON file
import json

config = {
    "model": "facebook/bart-large-mnli",
    "labels": ['5-10°C cold weather',
               '10-15°C cool weather',
               '15-20°C mild weather',
               '20-25°C warm weather',
               '25-30°C hot weather',
               'rain'],
    "hypothesis_template": "This item is comfortable to wear when the weather is {}."
}

with open("zero_shot_config.json", "w") as f:
    json.dump(config, f, indent=4)

In [58]:
files.download("zero_shot_config.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>